#### Moises Quiroz Diaz

##### Ejercicio 1

Utilizando las librerías requests y json , realize una consulta a la API BallDontLie https://
www.balldontlie.io/.

Mediante esta API, genere una consulta de 100 juegos (busquen el endpoint Get All Games
en la página, dentro de Games ) . Puede usar la opción ?per_page=100 en la dirección de la
API para lograr esto.

Guarde la consulta en una variable, y asegúrese que ésta sea semiestructurada. Consulte los
datos y metadatos asociados.

Dentro de la lista de datos, comente cuáles son las llaves de cada registro.

In [1]:
import requests
import json

In [2]:
r = requests.get('https://www.balldontlie.io/api/v1/games?per_page=100')

Se crea la variable objeto, la cual contiene una lista de 100 json los cuales corresponden a cada juego

In [11]:
objeto = r.json()['data']

Las llaves de los objetos son las siguientes, las cuales entregan los datos del partido, como la temporada y fecha del mismo, ademas cuenta con las llaves de 'home_team' y 'visitor_team' las cuales como valor contienen un json con la informacion del equipo como nombre y ciudad

In [24]:
list(dict(objeto[0]).keys())

['id',
 'date',
 'home_team',
 'home_team_score',
 'period',
 'postseason',
 'season',
 'status',
 'time',
 'visitor_team',
 'visitor_team_score']

##### Ejercicio 2: Organización de los datos

En base a los datos consultados en la API, genere un objeto pd.DataFrame con la siguiente
estructura:

Para obtener la estructura, sólo puede utilizar operaciones map y comprensiones de lista

In [26]:
import pandas as pd

In [62]:
def formato(entrada):
    datos = dict()
    datos['season'] = entrada['season']
    datos['period'] = entrada['period']
    datos['home'] = entrada['home_team']['full_name']
    datos['home_conference'] = entrada['home_team']['conference']
    datos['home_division'] = entrada['home_team']['division']
    datos['visitor'] = entrada['visitor_team']['full_name']
    datos['visitor_conference'] = entrada['visitor_team']['conference']
    datos['visitor_division'] = entrada['visitor_team']['division']
    datos['home_score'] = entrada['home_team_score']
    datos['visitor_score'] = entrada['visitor_team_score']
    datos['home_differencial'] = entrada['home_team_score'] - entrada['visitor_team_score']
    datos['visitor_differencial'] = -datos['home_differencial']
    return datos

In [72]:
columnas = ['season','period','home','home_conference','home_division','visitor','visitor_conference','visitor_division','home_score',
           'visitor_score','home_differencial','visitor_differencial']
dataframe = pd.DataFrame(list(map(formato,objeto)))[columnas]
dataframe.head(2)

,season,period,home,home_conference,home_division,visitor,visitor_conference,visitor_division,home_score,visitor_score,home_differencial,visitor_differencial
0,2018,4,Boston Celtics,East,Atlantic,Charlotte Hornets,East,Southeast,126,94,32,-32
1,2018,4,Boston Celtics,East,Atlantic,LA Clippers,West,Pacific,112,123,-11,11


#### Ejercicio 3
Genere una columna en el pd.DataFrame que identifique si el equipo que jugó de local ganó
(1) o no (0).

Repita el procedimiento para identificar si el equipo que jugó de visitante ganó (1) o no (0).

Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por jugar local.

Reporte cuáles son los primeros y últimos 5 equipos en cuanto a desempeño por jugar de visita.

In [90]:
dataframe['gano_local']=dataframe.apply(lambda x:int(x['home_differencial']> x['visitor_differencial']),axis=1)

In [93]:
dataframe[['home','visitor','gano_local']].head()

,home,visitor,gano_local
0,Boston Celtics,Charlotte Hornets,1
1,Boston Celtics,LA Clippers,0
2,Philadelphia 76ers,Denver Nuggets,1
3,Washington Wizards,Cleveland Cavaliers,1
4,Sacramento Kings,Miami Heat,1


In [95]:
dataframe['gano_visita']=dataframe.apply(lambda x:int(x['home_differencial']< x['visitor_differencial']),axis=1)

In [96]:
dataframe[['home','visitor','gano_visita']].head()

,home,visitor,gano_visita
0,Boston Celtics,Charlotte Hornets,0
1,Boston Celtics,LA Clippers,1
2,Philadelphia 76ers,Denver Nuggets,0
3,Washington Wizards,Cleveland Cavaliers,0
4,Sacramento Kings,Miami Heat,0


Los mejores 5 en rendimiento como local corresponden a:

In [111]:
dataframe.groupby(['home']).sum()[['gano_local']].sort_values('gano_local',ascending=False).head(5)

,gano_local
home,
Washington Wizards,4
Detroit Pistons,4
Indiana Pacers,4
Orlando Magic,4
Philadelphia 76ers,4


Los mejores 5 en rendimiento como visita corresponden a:

In [117]:
dataframe.groupby(['visitor']).sum()[['gano_visita']].sort_values('gano_visita',ascending=False).head(5)

,gano_visita
visitor,
Orlando Magic,4
LA Clippers,3
Miami Heat,2
Indiana Pacers,2
Portland Trail Blazers,2


##### Ejercicio 4: Obteniendo el porcentaje de ganar local y de visita

Genere un nuevo objeto que guarde el porcentaje de juegos ganados como local por equipo.

Repita lo mismo para los juegos donde el equipo fue visitante.

¿Qué equipos tienen iguales chances de ganar como visitante o local?

In [ ]:
lambda x: x['gano_local']/(x['gano_local']+x['gano_visita'])

In [125]:
ganados = dataframe.groupby(['home']).sum().apply(lambda x: x['gano_local']/(x['gano_local']+x['gano_visita']),axis=1)
ganados = pd.DataFrame(ganados)
ganados.columns=['% ganados como local']

In [131]:
ganados_v = dataframe.groupby(['visitor']).sum().apply(lambda x: x['gano_visita']/(x['gano_local']+x['gano_visita']),axis=1)
ganados_v = pd.DataFrame(ganados_v)
ganados_v.columns=['% ganados como visita']

In [141]:
ganados.head(5)

,% ganados como local
home,
Atlanta Hawks,0.200000
Boston Celtics,0.750000
Brooklyn Nets,0.333333
Charlotte Hornets,0.000000
Chicago Bulls,0.200000


In [142]:
ganados_v.head(5)

,% ganados como visita
visitor,
Atlanta Hawks,0.000000
Boston Celtics,0.666667
Brooklyn Nets,0.666667
Charlotte Hornets,0.285714
Chicago Bulls,0.250000


In [154]:
estadisticas = pd.concat([ganados,ganados_v],axis=1,sort=False)
estadisticas

,% ganados como local,% ganados como visita
Atlanta Hawks,0.200000,0.000000
Boston Celtics,0.750000,0.666667
Brooklyn Nets,0.333333,0.666667
Charlotte Hornets,0.000000,0.285714
Chicago Bulls,0.200000,0.250000
Cleveland Cavaliers,0.500000,0.000000
Dallas Mavericks,0.666667,0.000000
Denver Nuggets,1.000000,0.333333
Detroit Pistons,1.000000,0.500000
Golden State Warriors,0.750000,0.500000


Se observa un valor NaN en LA Clippers, esto debido a que este equipo solo ha jugado como visita

¿Qué equipos tienen iguales chances de ganar como visitante o local?

In [152]:
estadisticas[estadisticas['% ganados como local'] == estadisticas['% ganados como visita']]

,% ganados como local,% ganados como visita
Los Angeles Lakers,0.500000,0.500000
New Orleans Pelicans,0.333333,0.333333
New York Knicks,0.000000,0.000000
Phoenix Suns,0.000000,0.000000
